In [36]:
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import glob
import json

import cv2
import torch
from tqdm import tqdm_notebook
from tqdm import tqdm
from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader
import torchvision

In [ ]:
!git add .
!git commit -am "added 1st rank submissions by merging baseline w oc

In [37]:
META_DIR = "data/metadata/metadata/"
TRAIN_DIR = "data/train/train/"
TEST_DIR = "data/test/test/"

test_data = json.load(open(META_DIR + 'iwildcam2022_test_information.json'))
train_data = json.load(open(META_DIR + 'iwildcam2022_train_annotations.json'))

#Test images
df_test = pd.DataFrame({'id': [item['id'] for item in test_data['images']],
                                'seq_id': [item['seq_id'] for item in test_data['images']],
                                'file_name': [item['file_name'] for item in test_data['images']],
                                'location': [item['location'] for item in test_data['images']],
                                'seq_num_frames': [item['seq_num_frames'] for item in test_data['images']],
                                'seq_frame_num': [item['seq_frame_num'] for item in test_data['images']]})
#Train images
df_train = pd.DataFrame({'id': [item['id'] for item in train_data['images']],
                                'seq_id': [item['seq_id'] for item in train_data['images']],
                                'file_name': [item['file_name'] for item in train_data['images']],
                                'location': [item['location'] for item in train_data['images']],
                                'seq_num_frames': [item['seq_num_frames'] for item in train_data['images']],
                                'seq_frame_num': [item['seq_frame_num'] for item in train_data['images']]})
# Detection for train test
detections = json.load(open(META_DIR+"iwildcam2022_mdv4_detections.json"))['images']
det_dict = dict()
for detection in tqdm(detections):
    det_dict[detection['file']] = detection['detections']
df_detection = pd.DataFrame({'file': [item['file'] for item in detections],
                                'detections': [item['detections'] for item in detections]})
# Test sequence ids
test_sequence_ids = pd.unique(df_test['seq_id'])
# Train sequence id and count
train_seq_count = pd.read_csv(META_DIR+"train_sequence_counts.csv")

100%|██████████| 261428/261428 [00:00<00:00, 1260411.11it/s]


In [ ]:
from PIL import Image, ImageFile
from deep_sort.deep_sort.ocsort import OCSort

ImageFile.LOAD_TRUNCATED_IMAGES = True
file = open('submission_oc_det0.4.txt', 'w')
file.write("Id,Predicted")
counts = []
for index, seq_row in tqdm(train_seq_count.iterrows()) :
    #print("######################### NEW SEQ ##################")
    #print(seq_row["seq_id"])
    img_rows = df_train.loc[df_train.seq_id == seq_row["seq_id"]]
    tracker = OCSort(det_thresh=0.4, max_age=30, min_hits=3, 
        iou_threshold=0, delta_t=3, asso_func="iou", inertia=0.2, use_byte=False)
    #results = []
    outputs = []
    for index, img_row in img_rows.iterrows():
        image = Image.open(TRAIN_DIR+img_row['file_name'])
        detections = df_detection.loc[df_detection.file == ('train/'+img_row['file_name'])]['detections'].to_list()[0]
        dets = np.zeros((1, 4))
        if detections:
            dets = np.array([d["bbox"] for d in detections])
        cates = np.array([d["category"] for d in detections])
        confs = np.array([d["conf"] for d in detections])
        output = tracker.update_public(dets, cates, confs)
        #print("detections", detections)
        #print("return", output)
        if not output.size == 0:
            outputs.append(output[:,4].astype('float64').max())
        
    count = len(tracker.trackers)
    if not len(outputs) == 0:
        counts.append(max(outputs))
    else:
        counts.append(0)
    #print(count, max(outputs))
    file.write('\n' + str(seq_row["seq_id"]) + ',' + str(count))


1780it [05:27,  5.43it/s]


In [ ]:
from PIL import Image, ImageFile
from deep_sort.deep_sort.ocsort import OCSort
from deep_sort.application_util.preprocessing import non_max_suppression

ImageFile.LOAD_TRUNCATED_IMAGES = True
file = open('submission_oc_det0.5_sup.txt', 'w')
file.write("Id,Predicted")
counts = []
for index, seq_row in tqdm(train_seq_count.iterrows()) :
    #print("######################### NEW SEQ ##################")
    img_rows = df_train.loc[df_train.seq_id == seq_row["seq_id"]]
    tracker = OCSort(det_thresh=0.5, max_age=30, min_hits=3, 
        iou_threshold=0, delta_t=3, asso_func="iou", inertia=0.2, use_byte=False)
    #results = []
    for index, img_row in img_rows.iterrows():
        image = Image.open(TRAIN_DIR+img_row['file_name'])
        detections = df_detection.loc[df_detection.file == ('train/'+img_row['file_name'])]['detections'].to_list()[0]
        boxes = np.array([d["bbox"] for d in detections])
        scores = np.array([d["conf"] for d in detections])
        indices = non_max_suppression(boxes, 0.1, scores)
        detections = [detections[i] for i in indices]
        dets = np.zeros((1, 4))
        if detections:
            dets = np.array([d["bbox"] for d in detections])
        cates = np.array([d["category"] for d in detections])
        confs = np.array([d["conf"] for d in detections])
        output = tracker.update_public(dets, cates, confs)
        if not output.size == 0:
            outputs.append(output[:,4].astype('float64').max())
    #count = len(tracker.trackers)
    if not len(outputs) == 0:
        counts.append(max(outputs))
    else:
        counts.append(0)
    file.write('\n' + str(seq_row["seq_id"]) + ',' + str(count))
file.close()

347it [01:11,  5.62it/s]/home/briancy2/iwildcam2022/deep_sort/deep_sort/ocsort.py:42: RuntimeWarning: invalid value encountered in true_divide
  h = x[2] / w
1780it [05:32,  5.36it/s]


In [17]:
from PIL import Image, ImageFile
from deep_sort.deep_sort.ocsort import OCSort
ImageFile.LOAD_TRUNCATED_IMAGES = True
file = open('submission_oc_det0.5-2.txt', 'w')
file.write("Id,Predicted")
counts = []
i = 0
for index, seq_row in tqdm(train_seq_count.iterrows()) :
    #print("######################### NEW SEQ ##################")
    #print(seq_row["seq_id"])
    img_rows = df_train.loc[df_train.seq_id == seq_row["seq_id"]]
    tracker = OCSort(det_thresh=0.8, max_age=30, min_hits=3, 
        iou_threshold=0, delta_t=3, asso_func="iou", inertia=0.2, use_byte=False)
    trackers = []
    for index, img_row in img_rows.iterrows():
        image = Image.open(TRAIN_DIR+img_row['file_name'])
        detections = df_detection.loc[df_detection.file == ('train/'+img_row['file_name'])]['detections'].to_list()[0]
        dets = np.zeros((1, 5))
        detections = [d["bbox"] + [d["conf"]] for d in detections if d["conf"] < 0.95]
        if detections:
            dets = np.array(detections)
        output = tracker.update(dets, image.size, image.size)
        #print([t.id for t in tracker.trackers])
        trackers.extend(tracker.trackers)
    count = len(set([t.id for t in trackers]))
    #print(count)
    #print(results[i])                         
    #print(train_seq_count["count"].iloc[i])
    i += 1
    file.write('\n' + str(seq_row["seq_id"]) + ',' + str(count+results[i-1]))
file.close()

1780it [05:22,  5.51it/s]


In [42]:
from PIL import Image, ImageFile
from deep_sort.deep_sort.ocsort import OCSort
ImageFile.LOAD_TRUNCATED_IMAGES = True
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
file = open('submission_res93.txt', 'w')
file.write("Id,Predicted")
for sequence_id in tqdm(test_sequence_ids):
    d_count = []
    img_rows = df_test.loc[df_test.seq_id == sequence_id]

    for index, img_row in img_rows.iterrows():
        #image = Image.open(TEST_DIR+img_row['file_name'])
        detections = df_detection.loc[df_detection.file == ('test/'+img_row['file_name'])]['detections'].to_list()[0]
        d_count.append(len([d for d in detections if d["conf"] > 0.93]))
    count = max(d_count)
    file.write('\n' + str(sequence_id) + ',' + str(count))
file.close()    

100%|██████████| 11028/11028 [21:08<00:00,  8.69it/s]


In [22]:
from deep_sort.application_util import preprocessing
from deep_sort.application_util import visualization
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker
from img2vec_pytorch import Img2Vec
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
img2vec = Img2Vec(cuda=True, model='efficientnet_b7')

print(len(train_seq_count))
file = open('submission_deep_conf_0.5_eff.txt', 'w')
file.write("Id,Predicted")
counts = []
metric = nn_matching.NearestNeighborDistanceMetric("cosine", 0, 100)
i = 0
for index, seq_row in tqdm(train_seq_count.iterrows()) :
    tracker = Tracker(metric)
    r = []
    track_list = []
    img_rows = df_train.loc[df_train.seq_id == seq_row["seq_id"]]
    for index, img_row in img_rows.iterrows():
        detections = df_detection.loc[df_detection.file == ('train/'+img_row['file_name'])]['detections'].to_list()[0]
        image = Image.open(TRAIN_DIR+img_row['file_name'])
        #print(detections[0]["bbox"])
        #x = detections[0]["bbox"][1]+detections[0]["bbox"][0]
        #detections[0]["bbox"][0]:x
        #print(x)
        #print(image[0:int(image.shape[1]*x),3:5,:])
        #,detections[0]["bbox"][2]-detections[0]["bbox"][3]:detections[0]["bbox"][2],:
        detections = [Detection(tuple(det["bbox"]), det["conf"], []) for det in detections if det["conf"] < 0.95]
#         boxes = np.array([d for d in filtered_det])
#         scores = np.array([d.confidence for d in detections])
        tracker.predict()
        tracker.update(detections)
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            r.append(track.track_id)
    count = len(set(r)) + results[i]
    counts.append(count)
    file.write('\n' + str(seq_row["seq_id"]) + ',' + str(count))
    i += 1
file.close()    

Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-dcc49843.pth" to /home/briancy2/.cache/torch/hub/checkpoints/efficientnet_b7_lukemelas-dcc49843.pth
100%|██████████| 255M/255M [00:02<00:00, 96.7MB/s] 


1780


1780it [24:42,  1.20it/s]


In [23]:
loss = np.mean(np.abs((np.array(counts) - train_seq_count["count"].to_numpy())))

In [24]:
print(loss)

1.2915730337078652


In [25]:
print(np.asarray(np.unique((np.array(counts) - train_seq_count["count"]), return_counts=True)).T)

[[ -7   4]
 [ -6  15]
 [ -5  24]
 [ -4  36]
 [ -3  49]
 [ -2  95]
 [ -1 211]
 [  0 680]
 [  1 322]
 [  2 166]
 [  3  89]
 [  4  43]
 [  5  26]
 [  6   9]
 [  7   2]
 [  8   7]
 [  9   1]
 [ 13   1]]


In [51]:
loss = np.mean(np.abs(np.array(np.minimum(counts, np.array(results))) - train_seq_count["count"]))

In [38]:
train_seq_count["count"].iloc[0]

9

In [12]:
print(np.mean(np.abs(results - train_seq_count["count"])))

1.0016853932584269


In [13]:
print(loss)

1.0016853932584269


In [14]:
print(np.asarray(np.unique((np.array(results) - train_seq_count["count"]), return_counts=True)).T)

[[ -7  12]
 [ -6  48]
 [ -5  44]
 [ -4  49]
 [ -3  62]
 [ -2 166]
 [ -1 384]
 [  0 930]
 [  1  77]
 [  2   8]]


In [3]:
print(len(counts))

1780


In [26]:
counts_oc = pd.read_csv('submission_oc_det0.5-2.txt')["Predicted"].to_numpy()
print(np.mean(np.abs((np.array(counts_oc) - train_seq_count["count"].to_numpy()))))
print(np.asarray(np.unique((np.array(counts_oc) - train_seq_count["count"]), return_counts=True)).T)

0.9629213483146067
[[ -7   2]
 [ -6   7]
 [ -5  22]
 [ -4  48]
 [ -3  38]
 [ -2  78]
 [ -1 235]
 [  0 703]
 [  1 482]
 [  2 133]
 [  3  25]
 [  4   7]]


In [34]:
print(np.asarray(np.unique(train_seq_count["count"], return_counts=True)).T)

[[  1 408]
 [  2 307]
 [  3 351]
 [  4 259]
 [  5 156]
 [  6  71]
 [  7  31]
 [  8  15]
 [  9 176]
 [ 10   6]]


In [32]:
true_vals = {}
n = np.abs((np.array(counts_oc) - train_seq_count["count"]))
for i, count in enumerate(train_seq_count["count"]):
    if count in true_vals.keys():
        true_vals[count] += n[i]
    else:
        true_vals[count] = 0
    

In [33]:
true_vals

{9: 520, 10: 6, 3: 185, 2: 192, 1: 299, 5: 137, 4: 208, 7: 45, 6: 86, 8: 20}

In [6]:
merged_results = np.minimum(counts_oc, np.array(results))
print(np.mean(np.abs(merged_results - train_seq_count["count"].to_numpy())))
print(np.asarray(np.unique((np.array(merged_results) - train_seq_count["count"]), return_counts=True)).T)

1.0910112359550561
[[ -8   1]
 [ -7  18]
 [ -6  51]
 [ -5  45]
 [ -4  47]
 [ -3  69]
 [ -2 190]
 [ -1 416]
 [  0 864]
 [  1  72]
 [  2   7]]


In [ ]:
merged_results = np.minimum(counts_oc, np.array(results))
print(np.mean(np.abs(merged_results - train_seq_count["count"].to_numpy())))
print(np.asarray(np.unique((np.array(merged_results) - train_seq_count["count"]), return_counts=True)).T)

In [33]:
print(np.array(results).shape, counts_oc.shape)


(1780,) (1780,)


In [16]:
counts_deep = pd.read_csv('submission_deep_conf_0.5_nn0.txt')["Predicted"].to_numpy()
print(np.mean(np.abs((np.array(counts_deep) - train_seq_count["count"].to_numpy()))))
print(np.asarray(np.unique((np.array(counts_deep) - train_seq_count["count"]), return_counts=True)).T)

1.3780898876404495
[[ -9   3]
 [ -8  20]
 [ -7  18]
 [ -6  36]
 [ -5  41]
 [ -4  58]
 [ -3  96]
 [ -2 167]
 [ -1 432]
 [  0 604]
 [  1 216]
 [  2  63]
 [  3  18]
 [  4   5]
 [  5   1]
 [  6   2]]


In [17]:
merged_deep = np.minimum(counts_deep, np.array(results))
print(np.mean(np.abs(merged_results - train_seq_count["count"].to_numpy())))
print(np.asarray(np.unique((np.array(merged_results) - train_seq_count["count"]), return_counts=True)).T)

1.1258426966292134
[[ -8   3]
 [ -7  21]
 [ -6  48]
 [ -5  47]
 [ -4  55]
 [ -3  69]
 [ -2 184]
 [ -1 432]
 [  0 845]
 [  1  69]
 [  2   7]]


In [40]:
counts_deep0 = pd.read_csv('submission_oc_det0.5.txt')["Predicted"].to_numpy()
print(np.mean(np.abs((np.array(counts_deep0) - train_seq_count["count"].to_numpy()))))
print(np.asarray(np.unique((np.array(counts_deep0) - train_seq_count["count"]), return_counts=True)).T)

1.4797752808988764
[[ -8   1]
 [ -7   8]
 [ -6  22]
 [ -5  29]
 [ -4  39]
 [ -3  65]
 [ -2  99]
 [ -1 257]
 [  0 645]
 [  1 331]
 [  2 117]
 [  3  62]
 [  4  24]
 [  5  22]
 [  6  20]
 [  7   8]
 [  8   9]
 [  9   7]
 [ 10   1]
 [ 11   4]
 [ 12   3]
 [ 14   2]
 [ 18   4]
 [ 29   1]]


In [11]:
loss = np.mean(np.abs((np.array(results) - train_seq_count["count"].to_numpy())))

In [12]:
print(loss)

1.6578651685393258


In [9]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
file = open('submission_mdv40.95.txt', 'w')
file.write("Id,Predicted")
results = []
for index, seq_row in tqdm(train_seq_count.iterrows()) :
    base_counts = []
    img_rows = df_train.loc[df_train.seq_id == seq_row["seq_id"]]
    for index, img_row in img_rows.iterrows():
        #image = Image.open(TRAIN_DIR+img_row['file_name'])
        detections = df_detection.loc[df_detection.file == ('train/'+img_row['file_name'])]['detections'].to_list()[0]
        l = [d for d in detections if d["conf"] > 0.95]
        count=len(l)
        base_counts.append(count)
    results.append(max(base_counts))
    file.write('\n' + str(seq_row["seq_id"]) + ',' + str(base_counts))
file.close()    

1780it [04:34,  6.48it/s]
